In [2]:
import requests
import psycopg2
import datetime
import aiohttp
import asyncio
from time import sleep
import nest_asyncio
nest_asyncio.apply()

In [3]:
#This Class contains all main functions
class Common:
    #Protection from SQL injections 
    def SqlEncode(value):
        return "'" + str(value).replace( "'", "''" ) + "'"
    
    #Executing DML queries
    def SqlExecuteNonQuery(sql):
        conn = psycopg2.connect(
            database = "airflow"
            , user = "airflow"
            , password = "airflow"
            , host = "localhost"
            , port = "5432"
        )
        
        cursor = conn.cursor()
        cursor.execute(sql)
        conn.commit()
        cursor.close()
        conn.close()
        
        

In [6]:
async def fetch(c, url):
    tt = 2
    headers = {
        'User-Agent': 'Other'
      }
    while True:
        #print(url['url'])
        response = await c.get(url['url'], headers=headers)
        #If status_code is not 429 then status is "done"
        if response.status_code != 429:
            url['result'] = response.status_code
            url['status'] = 'done'
            url['response'] = response
            break
        #If status_code is 429 then the program sleeps 2 seconds
        else:
            sleep(tt)
    return url

In [7]:
async def fetch_all(c, urls):
    tasks = []
    urlTasks = [{'url': i, 'result': None, 'status': 'new', 'response' : None} for i in urls]
    #print(urlTasks)
    for url in urlTasks:
        task = asyncio.create_task(fetch(c,url))
        tasks.append(task)
    responses = await asyncio.gather(*tasks)
    
    return responses

In [ ]:
#List of html pages
htmlList = []
t = 1
#Processing one hundred pages in one cycle
for i in range(t, 1300):
    urlList = ["https://howlongtobeat.com/game/" + str(i) for i in range(t, i*100 + 1) ] #range(1, 200 + 1) [38000] range(t, i * 2000 + 1)
    #print(url)
    print(urlList[0])
    print(urlList[-1])
    status = 'In Progress'
    t = i * 100 + 1
    while status == 'In Progress':
        try:
            async with httpx.AsyncClient() as c:
            #r = requests.get(url, headers = headers)
                responseList = await fetch_all(c, urlList)

            for r in responseList:
                if r['response'].status_code == 200:
                    htmlList.append(r['response'].text)
            print(len(htmlList))
            status = 'Done'
        except:
            #If anything goes wrong then the programm sleeps 30 seconds
            sleep(30)
            print('sleep 30')
    sleep(5)

In [10]:
#Preparing rows to insert into DB
rowList = []
for htmlString in htmlList:
    try:
        if htmlString.find("<h4>Main Story</h4><h5>") >= 1:
            
            urlPre = htmlString[htmlString.find('<link rel="canonical" href="') + 28:]
            url = urlPre[:urlPre.find('"')]
            
            #Getting a main story duration
            mainStoryPre = htmlString[htmlString.find("<h4>Main Story</h4><h5>") + 23:]
            mainStory = mainStoryPre[:mainStoryPre.find("<")].replace('½', '.5')
            duration = float(mainStory[:mainStory.find(" ")])
            duration

            #Getting a game name
            gameNamePre = htmlString[htmlString.find("<title>How long is ") + 19:]
            gameName = gameNamePre[:gameNamePre.find("?")]
            gameName

            #Getting a release date
            releaseDatePre1 = htmlString[htmlString.find( "EU:</strong>" ):]
            releaseDatePre = releaseDatePre1[releaseDatePre1.find( "<br/>" ) + 5:]
            releaseDateText = releaseDatePre[:releaseDatePre.find( "<" )]

            #Getting month from the string
            monthText = releaseDateText[:releaseDateText.find( " " )]
            #Transforming a month name to a month number
            month = datetime.datetime.strptime(monthText, '%B').month

            if (releaseDateText.find( "," ) > 0):
                #Getting day from the string
                day = releaseDateText[releaseDateText.find( " " ) + 1:releaseDateText.find( "," )]

                #Getting year from the string
                year = releaseDateText[releaseDateText.find( "," ) + 2:]

                #Forming date in the database format
                releaseDate = "'" + year + "-" + str(month) + "-" + day + "'"
                releaseDate


                #Getting platforms
                platformPre1 = htmlString[htmlString.find( "<strong>Platform" ):]
                platformPre = platformPre1[platformPre1.find( "<br/>" ) + 5:]
                platform = platformPre[:platformPre.find( "<" )]
                platform


                row = (gameName, platform, duration, releaseDateText, releaseDate, url)
                rowList.append( row )
    except:
        x = 1

In [12]:
#Forming INSERT string
sqlString = "insert into hltb.hltb_games(game_name, platform, duration, release_date_text, release_date, url) VALUES"

#Common.SqlEncode is used to avoid SQL injections
for row in rowList:
    sqlString = sqlString + " (" + Common.SqlEncode(row[0]) + ", " + Common.SqlEncode(row[1])+ ", " + Common.SqlEncode(row[2])+ ", " + Common.SqlEncode(row[3])+ ", " + Common.SqlEncode(row[4])+ ", " + Common.SqlEncode(row[5])+ "),"

In [ ]:
#Removing a comma in the end of the string
sqlString = sqlString[:-1]
sqlString

In [14]:
#Executing SQL
Common.SqlExecuteNonQuery(sqlString)

#### 